<a href="https://colab.research.google.com/github/dmarinere/NikeCampaign/blob/master/Nike_Campaign_Contd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.cluster import KMeans


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
df_infl = pd.read_csv("/content/drive/My Drive/Colab Notebooks/all_influencers_tweets.csv")
df_gov = pd.read_csv("/content/drive/My Drive/Colab Notebooks/all_government_tweets.csv")

In [4]:
#i would start would creating a new dataframe that has no duplicate author so i could get some data
df_new_infl= df_infl.drop_duplicates(subset ="original_author",)
df_new_gov = df_gov.drop_duplicates(subset ="original_author")
df_new_gov.reset_index(drop=True, inplace=True)
df_new_infl.reset_index(drop=True, inplace=True)

In [392]:
#i started with using the assumption that we made about less than 5 tweets
df_infl = df_infl[df_infl['original_author'].map(df_infl['original_author'].value_counts()) > 5]
df_gov = df_gov[df_gov['original_author'].map(df_gov['original_author'].value_counts()) > 5]

In [396]:
# we would drop rows that the followers and following are missing and also original author 
df_new_infl = df_new_infl.dropna(how='any', subset=['followers', 'following', 'original_author'])
df_new_infl = df_new_infl.dropna(how='any', subset=['followers', 'following', 'original_author'])

In [397]:
#i converted some columns from object to int so we can work with them easily
df_new_infl = df_new_infl.astype({'favorite_count': 'int64', 'retweet_count': 'int64','following': 'int64' })
df_infl = df_infl.astype({'favorite_count': 'int64', 'retweet_count': 'int64','following': 'int64' })


In [398]:
# i would start with getting the reach score for all the influencers and leaders 
df_new_gov['reach'] = df_new_gov.apply(lambda x: x['followers'] - x['following'], axis=1)
df_new_infl['reach'] = df_new_infl.apply(lambda x: x['followers'] - x['following'], axis=1)

In [399]:

#we have been able to succesfully get the reach of all the influencer i would go ahead to sort each of them by reach
df_new_infl =df_new_infl.sort_values(by='reach', ascending=False)
df_new_gov=df_new_gov.sort_values(by="reach", ascending=False)
df_new_gov.reset_index(drop=True, inplace=True)
df_new_infl.reset_index(drop=True, inplace=True)

In [400]:
gov_name = {}
for name in range(len(df_gov_rank)):
     gov_name[df_gov_rank['original_author'][name]]= df_gov_rank['name'][name]

In [401]:
infl_name = {}
for name in range(len(df_infl_rank)):
     infl_name[df_infl_rank['original_author'][name]]= df_infl_rank['name'][name]

In [402]:
df_gov_rank = df_new_gov[['original_author','name','followers', 'following','reach']]
df_infl_rank = df_new_infl[['original_author','name','followers', 'following','reach']]

In [403]:
#rank them by reach
df_gov_rank["Rank: By Reach"] =df_gov_rank['reach'].rank(ascending=False)
df_infl_rank["Rank: By Reach"] = df_infl_rank['reach'].rank(ascending=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [404]:
df_gov_rank.head()

,original_author,name,followers,following,reach,Rank: By Reach
0,MBuhari,Muhammadu Buhari,3268751,26,3268725,1.0
1,PaulKagame,Paul Kagame,1981691,181,1981510,2.0
2,KagutaMuseveni,Yoweri K Museveni,1810417,28,1810389,3.0
3,PresidencyZA,Presidency | South Africa 🇿🇦,1597615,14,1597601,4.0
4,NAkufoAddo,Nana Akufo-Addo,1505263,352,1504911,5.0


In [405]:
df_infl_rank.head()

,original_author,name,followers,following,reach,Rank: By Reach
0,Trevornoah,Trevor Noah,10802767.0,325,10802442.0,1.0
1,News24,News24,3575205.0,632,3574573.0,2.0
2,Julius_S_Malema,Julius Sello Malema,3125926.0,652,3125274.0,3.0
3,GarethCliff,Gareth Cliff,1974502.0,356,1974146.0,4.0
4,euphonik,Euphonik™♛,1754199.0,65,1754134.0,5.0


### Cleaning and recovering the rank by number of retweets and likes

In [406]:
#since i didn't get all the tweets for each of them,  i focused on the sum of the top 100 tweets in the dataset 

In [409]:
#then i grouped tweet by author and sort them, then took only the top 100 retweets and like for each
#i sorted first by like because it is easier to get likes and those two work hand in hand
df_infl_popl = df_infl.sort_values(['favorite_count', 'retweet_count'], ascending=False).groupby('name').head(100)
df_gov_popl = df_gov.sort_values(['favorite_count', 'retweet_count'], ascending=False).groupby('name').head(100)

In [410]:
#Then i took only the columns that were important in this popularity analysis which is retweet and like
df_infl_popl = df_infl_popl[["name", "retweet_count", "favorite_count"]]
df_gov_popl = df_gov_popl[["name", "retweet_count", "favorite_count"]]

In [411]:
df_infl_popl = df_infl_popl.groupby(['name']).sum()
df_gov_popl = df_gov_popl.groupby(['name']).sum()


In [412]:
df_infl_popl= df_infl_popl.sort_values(["favorite_count", "retweet_count"], ascending=False).reset_index()
df_gov_popl= df_gov_popl.sort_values(["favorite_count", "retweet_count"], ascending=False).reset_index()

In [413]:
df_infl_popl["popularity"] = df_infl_popl['retweet_count'] + df_infl_popl["favorite_count"]
df_gov_popl["popularity"] = df_gov_popl['retweet_count'] + df_gov_popl["favorite_count"]

In [414]:
#here i sorted the values by popularity 
df_infl_popl =df_infl_popl.sort_values(["popularity"], ascending=False).reset_index(drop=True)
df_gov_popl =df_gov_popl.sort_values(["popularity"], ascending=False).reset_index(drop=True)

In [415]:
#here i did the ranking for both datasets 
df_infl_popl["Rank: By Popularity"] = df_infl_popl['popularity'].rank(ascending=False)
df_gov_popl["Rank: By Popularity"] = df_gov_popl['popularity'].rank(ascending=False)

In [416]:
df_infl_popl.head()

,name,retweet_count,favorite_count,popularity,Rank: By Popularity
0,Trevor Noah,526304,2945924,3472228,1.0
1,Mbuyiseni Ndlozi,42506,247513,290019,2.0
2,Euphonik™♛,23365,154250,177615,3.0
3,Ulrich Janse van Vuuren,17456,123550,141006,4.0
4,Helen Zille,15094,64536,79630,5.0


In [419]:
df_gov_popl.head()

,name,retweet_count,favorite_count,popularity,Rank: By Popularity
0,Muhammadu Buhari,289700,1434788,1724488,1.0
1,Nana Akufo-Addo,205035,1015494,1220529,2.0
2,Abdalla Hamdok,121221,562380,683601,3.0
3,Yoweri K Museveni,92523,496708,589231,4.0
4,Cyril Ramaphosa 🇿🇦 #StaySafe,93073,485854,578927,5.0


### Extract the top mentions 

In [420]:
#i created a new dataframe for mentions
df_mentions = df_infl['mention'].str.split(',', expand=True).rename(columns = lambda x: "string"+str(x+1))
df2_mention = df_gov['mention'].str.split(',', expand=True).rename(columns = lambda x: "string"+str(x+1))

In [421]:
#i melted all the values to a column since they are the same thing
new = pd.melt(df_mentions)
new2 = pd.melt(df2_mention)
#i dropped all null values
new = new.dropna(subset=['value'])
new2 = new2.dropna(subset=['value'])
#here i grouped by the number of time a value appeared
new = new.groupby('value').count().reset_index()
new2 = new2.groupby('value').count().reset_index()

In [422]:
#i did this to sort all the values in my data by number of mentions
new.sort_values(by='variable', ascending=False, inplace=True)
new2.sort_values(by='variable', ascending=False, inplace=True)

In [423]:
new.reset_index(drop=True, inplace=True)
new2.reset_index(drop=True, inplace=True)

In [424]:
#i created a list of all the accounts in my original dataset
names = list(df_new_infl['original_author'])
names2 = list(df_new_gov['original_author'])
#i used this to only get mentions of users in our influencers list which is the 
#scope of this project
mentions_infl =new[new['value'].isin(names)]
mentions_gov = new2[new2['value'].isin(names2)]
#i had to reset the index because a lot of rows were dropped
mentions_infl =mentions_infl.reset_index(drop=True)
mentions_gov = mentions_gov.reset_index(drop=True)
#i wanted the columns to have an understandable name 
mentions_infl=mentions_infl.rename(columns={'value' :'original_author',"variable": 'Count'})
mentions_gov=mentions_gov.rename(columns={'value' :'original_author',"variable": 'Count'})

In [425]:
mention_names =list(mentions_infl['original_author']) 
mention_names_gov = list(mentions_gov["original_author"])

In [426]:
df2 = pd.DataFrame({"original_author": mention_names,"Count":[0] * len(mention_names)}) 
df3= pd.DataFrame({"original_author": mention_names_gov, "Count": [0] * len(mention_names_gov)})
# for appending df2 at the end of df1 
mentions_infl =mentions_infl.append(df2, ignore_index=True) 
mentions_gov= mentions_gov.append(df3, ignore_index=True)

In [427]:
mentions_gov["name"] = mentions_gov['original_author'].map(gov_name)
mentions_infl['name'] = mentions_infl['original_author'].map(infl_name)
column_names = ["original_author", "name", "Count"]

mentions_gov = mentions_gov.reindex(columns=column_names)
mentions_infl = mentions_infl.reindex(columns=column_names)

In [428]:
#here i did the ranking for both datasets 
mentions_infl["Rank: By Number of Mentions"] = mentions_infl['Count'].rank(ascending=False)
mentions_gov["Rank: By Number of Mentions"] = mentions_gov['Count'].rank(ascending=False)

In [430]:
mentions_gov.head()

,original_author,name,Count,Rank: By Number of Mentions
0,CyrilRamaphosa,Cyril Ramaphosa 🇿🇦 #StaySafe,190,1.0
1,Macky_Sall,Macky Sall,113,2.0
2,MagufuliJP,Dr John Magufuli,108,3.0
3,mohzambia,Ministry of Health Zambia,73,4.0
4,gouvbenin,Gouvernement du Bénin 🇧🇯,34,5.0


In [431]:
mentions_infl.head()

,original_author,name,Count,Rank: By Number of Mentions
0,_AfricanUnion,African Union,96,1.0
1,AfricaCheck,Africa Check,33,2.0
2,TheStar_news,The Star,27,3.0
3,TheAfricaReport,The Africa Report,20,4.0
4,BrendanSAfrica,Brendan Boyle,15,5.5


### Extract the top 5 unique hashtags these influencers and government officials used in their top 10 retweets.  Group them based on their top 5 hashtags similarities. 

In [ ]:
#i want to start start with recovering the top ten retweets, 
#this signifies what their followers accept and support tells us a little bit about their audience 

In [432]:
#i would start from retrieving their top ten retweets for each 
#of the influencers and government official
#this code will first group the objects by name then sort all the values 
#by the number of retweets, then finally retrieve the first 10 rows for our use
df_infl_retweet = df_infl.sort_values(['retweet_count'], ascending=False).groupby('name').head(10)
df_gov_retweet = df_gov.sort_values(['retweet_count'], ascending=False).groupby('name').head(10)

In [433]:
#this would create a new column for each new string of hashtags 
top_ten_hashtags_infl = df_infl_retweet['hashtags'].str.split(',', expand=True).rename(columns = lambda x: "hashtag"+str(x+1))
top_ten_hashtags_gov = df_gov_retweet['hashtags'].str.split(',', expand=True).rename(columns = lambda x: "hashtag"+str(x+1))

In [438]:
top_ten_hashtags_infl['original_author'] = df_infl_retweet['original_author']
top_ten_hashtags_gov['original_author'] = df_gov_retweet['original_author']


In [439]:
top_ten_hashtags_gov.dropna(thresh=2, inplace=True)
top_ten_hashtags_infl.dropna(thresh=2, inplace=True)

In [440]:
top_ten_hashtags_gov =top_ten_hashtags_gov.melt(id_vars='original_author')
top_ten_hashtags_infl= top_ten_hashtags_infl.melt(id_vars='original_author')

In [441]:
# i dropped all the null values, those without hashtags
gov = top_ten_hashtags_gov.dropna()
infl = top_ten_hashtags_infl.dropna()
infl['value'] = infl["value"].str.lower()
gov['value'] = gov['value'].str.lower()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [442]:
infl['value'] = infl['value'].str.replace(" ","")
gov['value'] = gov['value'].str.replace(" ","")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [445]:
infl= infl.drop(columns='variable')
gov = gov.drop(columns='variable')

In [446]:
hashtags_onehot = pd.get_dummies(infl[['value']], prefix="", prefix_sep="")
hashtags_gov_onehot = pd.get_dummies(gov[['value']], prefix="", prefix_sep="")


In [447]:
hashtags_onehot
# add original author column back to dataframe
hashtags_onehot['original_author'] = infl['original_author'] 
hashtags_gov_onehot['original_author'] = gov['original_author']
# move original author column to the first column
fixed_columns = [hashtags_onehot.columns[-1]] + list(hashtags_onehot.columns[:-1])
hashtags_onehot = hashtags_onehot[fixed_columns]
fixed_columns2 = [hashtags_gov_onehot.columns[-1]] + list(hashtags_gov_onehot.columns[:-1])
hashtags_gov_onehot = hashtags_gov_onehot[fixed_columns2]


In [448]:
#i grouped then and measure by the mean of the hashtags
hashtags_grouped = hashtags_onehot.groupby('original_author').mean().reset_index()
hashtags_gov_grouped = hashtags_gov_onehot.groupby('original_author').mean().reset_index()

In [449]:
hashtags_grouped.replace(0, np.nan, inplace=True)
hashtags_gov_grouped.replace(0, np.nan, inplace=True)

In [450]:
for hashtag in hashtags_grouped['original_author']:
    print("----"+hashtag+"----")
    temp = hashtags_grouped[hashtags_grouped['original_author'] == hashtag].T.reset_index()
    temp.columns = ['hashtag','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(5))
    print('\n')

----GarethCliff----
     hashtag  freq
0   hashtag5   0.5
1   hashtag6   0.5
2         a6   NaN
3  alshabaab   NaN
4        amg   NaN


----News24----
     hashtag  freq
0   hashtag4  0.55
1   hashtag3  0.36
2         a6  0.09
3  alshabaab   NaN
4        amg   NaN


----Trevornoah----
      hashtag  freq
0    hashtag1  0.35
1    hashtag2  0.24
2   alshabaab  0.06
3         amg  0.06
4  leadership  0.06


----euphonik----
     hashtag  freq
0   hashtag6   1.0
1         a6   NaN
2  alshabaab   NaN
3        amg   NaN
4   hashtag1   NaN




In [451]:
for hashtag in hashtags_gov_grouped['original_author']:
    print("----"+hashtag+"----")
    temp = hashtags_gov_grouped[hashtags_gov_grouped['original_author'] == hashtag].T.reset_index()
    temp.columns = ['hashtag','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(7))
    print('\n')

----a----
       hashtag  freq
0     hashtag4   1.0
1       bissau   NaN
2  coronavirus   NaN
3      covid19   NaN
4    covid19bj   NaN
5     covid_19   NaN
6       gambia   NaN


----p----
       hashtag  freq
0     hashtag1  0.50
1      covid19  0.15
2       bissau  0.05
3  coronavirus  0.05
4    covid19bj  0.05
5     covid_19  0.05
6       gambia  0.05


